# 04_Steam_Genres

---

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Row

### Chargement du dataset

In [0]:
df_steam_flat = spark.read.json("/dbfs/FileStore/export/steam_flat_prep.json")

---

### Analyse des genres

Quels sont les genres les plus représentés ?

In [0]:
df_steam_flat.select("genre").limit(5).toPandas()

,genre
0,"Indie, Simulation, Strategy"
1,"Adventure, Free to Play, Indie"
2,"Action, Adventure, Casual, Indie, Sports, Stra..."
3,"Action, Casual, Indie"
4,"Adventure, Free to Play, Indie"


Les genres sont stockés comme une chaine de caractère, chacun séparé par une virgule.

Je vais tout d'abord en faire une liste, puis j'explode cette colonne.

In [0]:
df_steam_genre = df_steam_flat \
  .withColumn("genre_list", F.split(F.col("genre"),",")) \
    .drop("genre")
df_steam_genre.select("genre_list").limit(5).toPandas()

,genre_list
0,"[Indie, Simulation, Strategy]"
1,"[Adventure, Free to Play, Indie]"
2,"[Action, Adventure, Casual, Indie, Sports,..."
3,"[Action, Casual, Indie]"
4,"[Adventure, Free to Play, Indie]"


In [0]:
df_steam_genre_exploded = df_steam_genre.withColumn("genre", F.explode("genre_list"))
df_steam_genre_exploded.select("genre").limit(10).toPandas()

,genre
0,Indie
1,Simulation
2,Strategy
3,Adventure
4,Free to Play
5,Indie
6,Action
7,Adventure
8,Casual
9,Indie


In [0]:
display(df_steam_genre_exploded.groupBy("genre").agg(F.countDistinct("appid").alias("nb_games")).orderBy(F.desc("nb_games")).limit(10).toPandas())

genre,nb_games
Indie,34123
Action,23168
Casual,12238
Adventure,11100
Adventure,9961
Strategy,9851
Casual,9720
Simulation,9449
RPG,8539
Early Access,6060


Databricks visualization. Run in Databricks to view.

Le genre le plus représenté est le genre **"Indie"** avec 34123 jeux, puis **"Action"** avec 23168 jeux.

---

Existe-il des genres qui ont un meilleur rapport critiques positives/négatives ?

J'additionne les critiques positives et negatives par genre

In [0]:
df_genre_reviews = df_steam_genre_exploded.groupBy("genre").agg(
    F.sum("positive").alias("sum_positive"),
    F.sum("negative").alias("sum_negative"))

Je calcule du ratio global positif/négatif par genre

In [0]:
display(df_genre_reviews.withColumn("review_ratio", F.col("sum_positive") / F.col("sum_negative")).orderBy(F.desc("review_ratio")).limit(10).toPandas())

genre,sum_positive,sum_negative,review_ratio
Photo Editing,572545,12430,46.061544650040226
Animation & Modeling,569505,12382,45.99458891939913
Design & Illustration,641338,18716,34.26683051934174
Utilities,659115,25808,25.539173899566027
Game Development,2335,119,19.6218487394958
Audio Production,53676,4196,12.792183031458531
Web Publishing,962,85,11.31764705882353
Indie,5445292,481604,11.306575526781339
Animation & Modeling,112426,11925,9.42775681341719
Video Production,101238,11556,8.760643821391485


Databricks visualization. Run in Databricks to view.

Les genres qui ont le meilleur ratio critiques positives/négatives sont **Photo Editing** et **Animation & Modeling**

---

Est-ce que certains éditeurs ont des genres favoris ?

Nombre de jeux par éditeur et par genre

In [0]:
df_pub_genre = df_steam_genre_exploded.groupBy("publisher", "genre").agg(F.count("appid").alias("nb_games"))

Nombre total de jeux par éditeur

In [0]:
df_pub_total = df_steam_genre_exploded.groupBy("publisher").agg(F.count("appid").alias("total_games"))

Calcul de la part de chaque genre chez l'éditeur

In [0]:
df_pub_pref = df_pub_genre.join(df_pub_total, on="publisher").withColumn("genre_share", F.col("nb_games") / F.col("total_games"))

On garde le genre le plus représenté chez un éditeur

In [0]:
from pyspark.sql import Window

window = Window.partitionBy("publisher").orderBy(F.desc("nb_games"))

display(df_pub_pref.withColumn("rank", F.row_number().over(window)).filter(F.col("rank") == 1).drop("rank").orderBy(F.desc("genre_share")).toPandas())

publisher,genre,nb_games,total_games,genre_share
Droid Riot,Action,1,1,1.0
Golden Dragon Studio,Casual,1,1,1.0
Maciej Targoni,Indie,1,1,1.0
Metal Fox,Indie,1,1,1.0
RoBot,Action,1,1,1.0
WhisperGames,Adventure,1,1,1.0
--,Casual,1,1,1.0
0o0,Audio Production,1,1,1.0
111144447777a,Action,1,1,1.0
11PLANNING,RPG,1,1,1.0


C'est assez difficile à dire car certains éditeurs sont très peu actifs et ont peu de jeux.

L'éditeur **8floor** a principalement des jeux Casual : 79% de ses 243 jeux.

L'éditeur **Big Fish Games** a principalement des jeux Adventure : 48% de ses 834 jeux.

---

Quels sont les genres les plus lucratifs ?

J'étudie la colonne `discount`

In [0]:
from pyspark.sql.functions import min, max

df_steam_genre_exploded.select(min("discount").alias("min_discount"), max("discount").alias("max_discount")).show()

+------------+------------+
|min_discount|max_discount|
+------------+------------+
|         0.0|        90.0|
+------------+------------+



`discount` semble être un pourcentage.

J'étudie la colonne `owners`

In [0]:
df_steam_genre_exploded.select("owners").distinct().show()

+--------------------+
|              owners|
+--------------------+
|  100,000 .. 200,000|
|1,000,000 .. 2,00...|
|20,000,000 .. 50,...|
|5,000,000 .. 10,0...|
|         0 .. 20,000|
|    20,000 .. 50,000|
|2,000,000 .. 5,00...|
|50,000,000 .. 100...|
|500,000 .. 1,000,000|
|10,000,000 .. 20,...|
|  200,000 .. 500,000|
|   50,000 .. 100,000|
|200,000,000 .. 50...|
+--------------------+



`owners` est une plage de valeurs, je vais donc utiliser la moyenne pour mes calculs

In [0]:
from pyspark.sql.functions import split, translate

owners_clean = translate(F.col("owners"), ",", "")
split_owners = split(owners_clean, " .. ")

df_steam_genre_exploded = df_steam_genre_exploded \
       .withColumn("owners_min", split_owners.getItem(0)) \
       .withColumn("owners_max", split_owners.getItem(1)) \
       .withColumn("owners_avg", (F.col("owners_min") + F.col("owners_max")) / 2)

df_steam_genre_exploded.select("owners_avg").limit(5).toPandas()

,owners_avg
0,150000.0
1,150000.0
2,150000.0
3,10000.0
4,10000.0


Calcul du coût total : prix * nombre d'utilisateurs

In [0]:
df_steam_genre_exploded = df_steam_genre_exploded.withColumn("total_price", F.col("owners_avg") * F.col("price"))

df_steam_genre_exploded.select("total_price").limit(5).toPandas()

,total_price
0,29850000.0
1,29850000.0
2,29850000.0
3,0.0
4,0.0


Calcul du prix avec la remise

In [0]:
df_steam_genre_exploded = df_steam_genre_exploded.withColumn("discount_price", \
                    (100 - F.col("discount")) * F.col("total_price") / 100)

df_steam_genre_exploded.select("discount_price").limit(5).toPandas()

,discount_price
0,5970000.0
1,5970000.0
2,5970000.0
3,0.0
4,0.0


Total des prix par genre

In [0]:
display(df_steam_genre_exploded.groupBy("genre").agg(F.sum("discount_price").alias("discount_price_sum")).orderBy(F.desc("discount_price_sum")).limit(10).toPandas())

genre,discount_price_sum
Action,3.4115663809E12
Indie,2.2435447303E12
Adventure,1.64381710475E12
RPG,1.18136506835E12
Simulation,1.1517550709E12
Strategy,9.860631986E11
Adventure,7.2471420365E11
Indie,5.303221544E11
Early Access,4.834405423E11
Casual,4.320073469E11


Databricks visualization. Run in Databricks to view.

Le genre **Action** est le plus lucratif, suivi de **Indie** et **Adventure**.